In [ ]:
import sympy as sp
from numpy import array
import numpy as np
from sympy import init_printing
init_printing(use_latex='mathjax')
import scipy


sqrt2 = np.sqrt(2)
sqrt3 = np.sqrt(3)
upper_triangle_idx  =np.triu_indices(5)

In [ ]:

def remove_hydrostatic_dependence(data, transpose_data):
    # Get the rotation matrix with full precision


    d = 1./np.sqrt(6) * np.array( [[ sqrt2, sqrt2, sqrt2, 0, 0, 0  ],
                                   [ -1, -1, 2, 0, 0, 0],
                                   [-sqrt3, sqrt3, 0, 0, 0, 0],
                                   [0, 0, 0, 2*sqrt3, 0, 0],
                                   [0, 0, 0, 0, 2*sqrt3, 0],
                                   [0, 0, 0, 0, 0, 2*sqrt3] ])

    # Isolate stress
    stress_vectors = data[:,0:6]
    stress_vectors_T = transpose_data[:, 0:6]

    # # Shift the X_0 up a tiny bit
    # data[:,6] += 1e-6
    # transpose_data[:,6] += 1e-6

    # Rotate to natural basis
    stress_vectors = (d @ stress_vectors.T).T
    stress_vectors_T = (d @ stress_vectors_T.T).T
    #print(np.shape(stress_vectors_T), np.shape(transpose_data[:,3].T))

    # We assert that there should be hydrostatic stress independence. Therefore, we only solve for the 5x5 submatrix
    data = np.column_stack([stress_vectors[:,1:6], data[:,6].T ])
    transpose_data = np.column_stack( [stress_vectors_T[:, 1:6], transpose_data[:,6].T ])

    return data, transpose_data

In [ ]:
# load data
hill_data_path = "../data_6x6/processed_data/transverse_iso_6x6_bingo.csv"
hill_transposed_data_path = "../data_6x6/processed_data/transverse_iso_6x6_bingo_transpose.csv"
data_pre = np.loadtxt(hill_data_path)
transposed_data_pre = np.loadtxt(hill_transposed_data_path)

In [ ]:
data, transposed_data = remove_hydrostatic_dependence(data_pre, transposed_data_pre)
eqps_0_data = np.isclose(data[:,5],0.0)
#print(eqps_0_data)
data = data[eqps_0_data,:]
eqps_0_trans_data = np.isclose(transposed_data[:,5],0.0)
transposed_data = transposed_data[eqps_0_trans_data, :]
data, transposed_data

In [ ]:
def objective(params):
    #print(params)
    # fill a P matrix that has the params:
    candidate_P = np.array( [[ params[0], params[1], 0, 0, 0 ],
                             [ params[1], params[2], 0, 0, 0],
                             [ 0, 0, params[3], 0, 0],
                             [ 0, 0, 0, params[4], 0],
                             [0, 0, 0, 0, params[5] ]
                             ] )
    #print(candidate_P)
    residual = np.zeros(len(data))
    for i, stress_vec in enumerate(data):
        sv = stress_vec[0:5]
        current_ys =  sv @ candidate_P @ sv 
        error = current_ys - 100

        residual[i] = error
        
    #print(scipy.linalg.norm(residual))
    return np.sum(np.abs(residual))
    


In [ ]:
initial_guess = np.array([ 9/2, 0.0, 31/2, 2, 2, 1.0 ])
result = scipy.optimize.minimize(objective, initial_guess, method = 'SLSQP', options={'fatol':1e-6, 'maxiter':1e5})
print(result)

In [ ]:
params = result.x
candidate_P = np.array( [[ params[0], params[1], 0, 0, 0 ],
                             [ params[1], params[2], 0, 0, 0],
                             [ 0, 0, params[3], 0, 0],
                             [ 0, 0, 0, params[4], 0],
                             [0, 0, 0, 0, params[5] ]
                             ] )
#print(candidate_P)
residual = np.zeros(len(data))
for i, stress_vec in enumerate(data):
    sv = stress_vec[0:5]
    current_ys =  sv @ candidate_P @ sv 
    error = current_ys - 100

    residual[i] = error
print(candidate_P)
print(residual)